In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
import sklearn.decomposition as decomposition
from sklearn.decomposition import SparsePCA
import sklearn.preprocessing as preprocessing
from sklearn.feature_selection import SelectKBest
import pandas as pd
import sklearn.preprocessing as pp
import sklearn.naive_bayes as naive_bayes
import sklearn.cross_validation as cross_validation

# Loading the Data

In [33]:
X = pd.read_csv('../assignment1_2016S1/training_data.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]
y = pd.read_csv('../assignment1_2016S1/training_labels.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]

# Feature Engineering / Pre-Processing

In [34]:
X_scaled = pp.MaxAbsScaler().fit_transform(X)

In [35]:
svd = decomposition.TruncatedSVD(n_components=700,random_state=42)

In [36]:
pca = decomposition.PCA(n_components=0.90)

In [ ]:
count = 0;
for name,series in X.iteritems():
    print(series.value_counts())

0.000000    20094
0.112368        1
0.382446        1
0.086506        1
0.147293        1
0.089342        1
0.787931        1
0.068985        1
0.067282        1
0.114734        1
0.063370        1
Name: 1, dtype: int64
0.000000    20097
0.094775        2
0.036499        1
0.137539        1
0.074199        1
0.028196        1
0.093208        1
Name: 2, dtype: int64
0.000000    20057
0.060121        3
0.020368        2
0.045882        2
0.084636        2
0.026989        1
0.113954        1
0.087175        1
0.038235        1
0.185479        1
0.101366        1
0.021739        1
0.051890        1
0.056242        1
0.072045        1
0.025791        1
0.124536        1
0.094755        1
0.060538        1
0.223526        1
0.032050        1
0.039625        1
0.038744        1
0.133092        1
0.023001        1
0.060962        1
0.311339        1
0.054146        1
0.068105        1
0.082241        1
0.073877        1
0.067058        1
0.025945        1
0.341863        1
0.026906        1
0.

(11, 13626)

# Gaussian Naive Bayes

In [174]:
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X, y, test_size=0.33, random_state=42)

nb = naive_bayes.GaussianNB()
mnb = naive_bayes.MultinomialNB()

raw_pipe = Pipeline([("naive_bayes",nb)])
pca_pipe = Pipeline([("pca",pca),("naive_bayes", nb)])
svd_pipe = Pipeline([("svd",svd),("naive_bayes",nb)])
raw_pipe_mnb = Pipeline([("naive_bayes",mnb)])
pipes = [raw_pipe,pca_pipe,svd_pipe,raw_pipe_mnb]
for pipeline in pipes:
    print(pipeline.fit(X_train,y_train.as_matrix().ravel()).score(X_test,y_test))

0.439733494852
0.199878861296
0.218655360388
0.560266505148


# Multinomial Naive Bayes

In [88]:

pipeline = Pipeline([(("multinomial_naive_bayes", mnb)])
pipeline.fit(X_train,y_train.as_matrix().ravel())
pipeline.score(X_test,y_test)

ValueError: Input X must be non-negative

# Decision Tree

# SVM - Linear Kernel

# Random Forest

In [3]:
# Author: Andreas Mueller <amueller@ais.uni-bonn.de>
#
# License: BSD 3 clause

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

iris = load_iris()

X, y = iris.data, iris.target

# This dataset is way to high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1 
[CV]  features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1, score=0.960784 -   0.0s
[CV] features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1 
[CV]  features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1, score=0.901961 -   0.0s
[CV] features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1 
[CV]  features__univ_select__k=1, svm__C=0.1, features__pca__n_components=1, score=0.979167 -   0.0s
[CV] features__univ_select__k=1, svm__C=1, features__pca__n_components=1 
[CV]  features__univ_select__k=1, svm__C=1, features__pca__n_components=1, score=0.941176 -   0.0s
[CV] features__univ_select__k=1, svm__C=1, features__pca__n_components=1 
[CV]  features__univ_select__k=1, svm__C=1, features__pca__n_components=1, score=0.921569 -   0.0s
[CV] features__univ_select__k=1, svm__C=1, features__pca__n_co

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  24 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:    0.2s finished
